In [1]:
using CSV, DataFrames, Gadfly, Distributions, StatsBase

┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly C:\Users\Antoine Lamontagne\.julia\packages\Gadfly\09PWZ\src\mapping.jl:228


In [2]:
train = CSV.read("ODtrain.csv")

,IPERE,Menage,M_AUTO,M_PERS,M_DOMSM,M_revenu,M_dist_train,M_dist_metro,M_dist_centre,M_emploiTC,P_RANG,P_SEXE,P_AGE,P_GRAGE,P_STATUT,P_PERMIS,D_HREDE,D_MOTIF,D_Mode
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,9,1000013,2,3,110,6,7.71979,1.81961,9.44328,556670,2,1,47,10,1,1,730,1,1
2,17,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,1,1,46,10,3,1,630,4,2
3,21,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,3,1,47,10,1,2,500,1,2
4,31,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,7,1,21,5,3,2,700,4,2
5,29,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,6,2,54,11,1,1,630,1,1
6,25,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,5,1,56,12,1,1,600,1,1
7,23,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,4,1,63,13,1,2,700,1,4
8,19,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,2,2,15,4,3,5,750,4,4
9,39,1000016,0,1,406,2,3.12452,0.689126,12.9415,658600,1,1,32,7,1,1,830,1,2


In [3]:
dropmissing!(train, disallowmissing=true)

,IPERE,Menage,M_AUTO,M_PERS,M_DOMSM,M_revenu,M_dist_train,M_dist_metro,M_dist_centre,M_emploiTC,P_RANG,P_SEXE,P_AGE,P_GRAGE,P_STATUT,P_PERMIS,D_HREDE,D_MOTIF,D_Mode
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,9,1000013,2,3,110,6,7.71979,1.81961,9.44328,556670,2,1,47,10,1,1,730,1,1
2,17,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,1,1,46,10,3,1,630,4,2
3,21,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,3,1,47,10,1,2,500,1,2
4,31,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,7,1,21,5,3,2,700,4,2
5,29,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,6,2,54,11,1,1,630,1,1
6,25,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,5,1,56,12,1,1,600,1,1
7,23,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,4,1,63,13,1,2,700,1,4
8,19,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,2,2,15,4,3,5,750,4,4
9,39,1000016,0,1,406,2,3.12452,0.689126,12.9415,658600,1,1,32,7,1,1,830,1,2


In [4]:
n = size(train,1)
p = size(train,2)

19

In [5]:
str_mode = ["Voiture", "Transport Collectif", "Hybride", "Actif", "Autre"]

d = Array{String}(undef,n)

for i=1:length(str_mode)
   
    ind = train[:D_Mode] .== i
    d[ind] .= str_mode[i] 
    
end

train[:D_Mode_str] = d

categorical!(train, :D_Mode_str)

,IPERE,Menage,M_AUTO,M_PERS,M_DOMSM,M_revenu,M_dist_train,M_dist_metro,M_dist_centre,M_emploiTC,P_RANG,P_SEXE,P_AGE,P_GRAGE,P_STATUT,P_PERMIS,D_HREDE,D_MOTIF,D_Mode,D_Mode_str
,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Categorical…
1,9,1000013,2,3,110,6,7.71979,1.81961,9.44328,556670,2,1,47,10,1,1,730,1,1,Voiture
2,17,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,1,1,46,10,3,1,630,4,2,Transport Collectif
3,21,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,3,1,47,10,1,2,500,1,2,Transport Collectif
4,31,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,7,1,21,5,3,2,700,4,2,Transport Collectif
5,29,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,6,2,54,11,1,1,630,1,1,Voiture
6,25,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,5,1,56,12,1,1,600,1,1,Voiture
7,23,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,4,1,63,13,1,2,700,1,4,Actif
8,19,1000014,2,7,117,2,8.25894,2.35297,11.1385,318570,2,2,15,4,3,5,750,4,4,Actif
9,39,1000016,0,1,406,2,3.12452,0.689126,12.9415,658600,1,1,32,7,1,1,830,1,2,Transport Collectif


In [6]:
struct Occurences
    mode::Array{Int64}
    variables::Array{Array}
    columns::Array{Symbol}
    nCat::Array{Int64}
end

In [19]:
## COPIED FROM MAX VRAISEMBLANCE
k=5
struct TestSet
    train::DataFrame
    test::DataFrame
end

function getTestSet(initialSet::DataFrame)
    n = length(initialSet[:D_Mode])
    nTest = convert(Int64, round(0.20 * n))
    index = rand(0:n-nTest+1)
    return TestSet(vcat(initialSet[1:index-1, :], initialSet[index+nTest:n, :]), initialSet[index:index+nTest-1, :])
    
    a = [1,2,3]
    b= view(a, 1:2)
    c= vcat(view(a, 1:1), view(a, 3:3))
end
function validate(estimates::Array{Int64}, realValue::Array{Int64})
    n = length(realValue)
    nEach = [0, 0, 0, 0, 0]
    right = [0, 0, 0, 0, 0]
    for i=1:n
        nEach[realValue[i]] += 1
        if estimates[i] == realValue[i]
            right[realValue[i]] += 1
        end
    end
    
    out = ""
    for i=1:k
        #print(str_mode[i], " : ", right[i]/nEach[i]*100, "%\n" ) 
        out *= str_mode[i] * " : " * string(right[i]/nEach[i]*100) * "%\n"
    end
    #print("Total : ", sum(right)/n*100, "%\n" )
    tot = sum(right)/n*100
    out *= "Total : " * string(tot) * "%\n"
    return (total=tot, output=out)
end

validate (generic function with 1 method)

In [27]:
function findProb(occ::Occurences, variable::Int64, value::Int64)
    q = Array{Float64}(undef, length(occ.mode))
    for i=1:5
        q[i] = log(occ.variables[i, variable][value] + 1) - log(occ.mode[i] + length(occ.nCat[variable]))        
    end
    return q
end

function getOccurences(trainSet::DataFrame, col::Array{Symbol}, nCat::Array{Int64})
    mode = Array{Int64, 1}(undef, 5)
    for i=1:5
        mode[i] = size(trainSet[trainSet[:D_Mode] .== i, :], 1)
    end
    
    variables = Array{Array{Int64, 1}, 2}(undef, (5, length(col)))
    
    for i=1:5
        for j=1:length(col)
            variables[i, j] = Array{Int64, 1}(undef, nCat[j])
            for k=1:nCat[j]
                variables[i, j][k] = size(trainSet[(trainSet[:D_Mode] .== i) .& (trainSet[col[j]] .== k), :], 1)
            end
        end
    end
    return Occurences(mode, variables, col, nCat)
end

function filterMode(row::DataFrameRow, occ::Occurences)
    p = Array{Float64}(undef, 5)
    for i=1:5
       p[i] = log(occ.mode[i] + 1) - log(sum(occ.mode) + length(occ.mode)) 
    end 
    for i=1:length(occ.columns)
        x̃ = row[occ.columns[i]]
        p = p + findProb(occ, i, x̃)
    end
    
     _, ind = findmax(p)
    
    return ind
end

function JustDoItDontLetYourDreamsBeDreams(col::Array{Symbol}, nCat::Array{Int64})
    set = getTestSet(train)
    occ = getOccurences(set.train, col, nCat)
    sizeTest = size(set.test, 1)
    est = Array{Int64}(undef, sizeTest)
    println(occ)
    println()
    for i=1:sizeTest
        est[i] = filterMode(set.test[i, :], occ)
    end
    validate(est, set.test[:D_Mode])
end

JustDoItDontLetYourDreamsBeDreams (generic function with 2 methods)

In [31]:
JustDoItDontLetYourDreamsBeDreams(    
    [:P_PERMIS, :P_GRAGE, :D_MOTIF, :P_STATUT],
    [5, 16, 13, 8]
)

Occurences([28839, 11795, 1670, 6653, 4065], Array[[24393, 681, 2, 1, 3762] [0, 2338, 1262, 1469, 1868, 1417, 1855, 2537, 2944, 3451, 4162, 3057, 1631, 554, 184, 110] [23028, 0, 0, 5811, 0, 0, 0, 0, 0, 0, 0, 0, 0] [20808, 1617, 6067, 188, 123, 0, 35, 1]; [6798, 3699, 16, 0, 1282] [0, 66, 870, 2417, 1650, 854, 907, 900, 828, 863, 1043, 783, 440, 120, 39, 15] [6642, 0, 0, 5153, 0, 0, 0, 0, 0, 0, 0, 0, 0] [5863, 595, 5257, 28, 38, 0, 12, 2]; [1494, 103, 1, 1, 71] [0, 1, 50, 198, 240, 93, 115, 136, 166, 193, 239, 163, 63, 11, 0, 2] [1173, 0, 0, 497, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1100, 56, 506, 4, 2, 0, 2, 0]; [2063, 1039, 8, 2, 3541] [0, 1760, 1565, 796, 292, 208, 273, 260, 275, 301, 362, 326, 150, 65, 14, 6] [2324, 0, 0, 4329, 0, 0, 0, 0, 0, 0, 0, 0, 0] [1901, 296, 4421, 17, 12, 0, 6, 0]; [188, 523, 1, 1, 3352] [0, 1309, 1647, 979, 30, 12, 21, 11, 12, 13, 6, 13, 9, 3, 0, 0] [99, 0, 0, 3966, 0, 0, 0, 0, 0, 0, 0, 0, 0] [82, 12, 3962, 3, 5, 0, 1, 0]], Symbol[:P_PERMIS, :P_GRAGE, :D_MOTIF, :P_S

(total = 60.535646925688425, output = "Voiture : 79.85561571567402%\nTransport Collectif : 48.842917251051894%\nHybride : 0.0%\nActif : 0.11567379988432619%\nAutre : 84.24591738712776%\nTotal : 60.535646925688425%\n")

In [ ]:
train